#  Action Rules Discovery - Titanic Case Study

This notebook presents a new implemetation of several action rule mining algorithms. Action rule mining was first introduced in:
    
    Ras, Zbigniew W., and Alicja Wieczorkowska. "Action-rules: How to increase profit of a company." European Conference on Principles of Data Mining and Knowledge Discovery. Springer, Berlin, Heidelberg, 2000.

Action rule mining is an extension of the widely used task of learning classification rules. In addition to information expressed in a standard classification rules, action rule provide 'actionable' counterfactual information:

* Finds rules which are statistically valid with respect to data
* Action rule has to contain at least one  influenceable (actionable) attribute


## Example

For illustration, we will use the well-known  Titanic dataset. 

---
** STANDARD CLASSIFICATION RULE**:


    IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=NO (WITH SUPPORT 6% AND CONFIDENCE 60%)
---

This rule predicts chances of survival for a particular person. An action rule is an extended classification rule, which also says what that person should have done differently to increase the chance of survival. 
In the Titanic dataset, the passenger cannot influcence his or her age or gender, but can influence the passenger class and boarding place - these become the actionable attributes.

The metrics support and confidence can be used to show the quality of the rule. The rule says that 6% of passengers were females, boarded in Southampton, and traveling in third class (support). And 60% of them did not survive (confidence).

---
** ACTION RULE**:

    Women who traveled from Southampton in the third class would increase their chance for survival if they would change their boarding place to Cherbourg and paid extra money for the first class.

---
This advise provided by this rule is to women, who boarded to the third class in Southampton, is to upgrade the ticket to the first class and change the boarding place to Cherbourg.

## Required inputs
* Tabular dataset
* Specification of target class
* List of actionable attributes


## Installation of the package
        
    
    pip install -i https://test.pypi.org/simple/ actionrules-lukassykora
        
For mining candidate rules, the actionrules package uses the very fast pyFIM package.
This package is not availabe in the pypi index, but it can be easily installed following these instructions: http://www.borgelt.net/pyfim.html.

## Action rule mining Python interface

As the first step, the module needs to be imported:

In [1]:
import pandas as pd
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

### Instantiate model object and load data

All columns in the input dataset must be nominal. Note: numerical variables can be converted to nominal with  discretization (binning). 

In [2]:
dataFrame = pd.read_csv("data/titanic.csv", sep=";")
dataFrame.head()

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
0,1,<16.13336;32.10002),S,very high,1.0,female,1.0
1,2,<0.1667;16.13336),S,very high,1.0,male,1.0
2,3,<0.1667;16.13336),S,very high,1.0,female,0.0
3,4,<16.13336;32.10002),S,very high,1.0,male,0.0
4,5,<16.13336;32.10002),S,very high,1.0,female,0.0


In [3]:
dataFrame[dataFrame.Sex=="female"][dataFrame.Embarked == "S"][dataFrame.Survived == 0.0]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,ID,Age,Embarked,Fare,Pclass,Sex,Survived
2,3,<0.1667;16.13336),S,very high,1.0,female,0.0
4,5,<16.13336;32.10002),S,very high,1.0,female,0.0
286,287,<48.06668;64.03334),S,very high,1.0,female,0.0
365,366,<32.10002;48.06668),S,higher,2.0,female,0.0
369,370,<16.13336;32.10002),S,higher,2.0,female,0.0
381,382,<16.13336;32.10002),S,avg,2.0,female,0.0
382,383,NaN,S,avg,2.0,female,0.0
411,412,<32.10002;48.06668),S,avg,2.0,female,0.0
445,446,<16.13336;32.10002),S,avg,2.0,female,0.0
456,457,<48.06668;64.03334),S,higher,2.0,female,0.0


### Configuration options

For mining from the input dataset, the **fit**  method can be used. Internally,  ActionRulesDiscovery uses the PyFIM library for classification rules discovery.  The fit function has a number of options and parameters:

In [4]:
actionRulesDiscovery = ActionRulesDiscovery()
help(actionRulesDiscovery.fit)

Help on method fit in module actionrules.actionRulesDiscovery.actionRulesDiscovery:

fit(stable_antecedents: List[str], flexible_antecedents: List[str], consequent: str, conf: float, supp: float, desired_classes: List[str] = None, desired_changes: List[list] = None, is_nan: bool = False, is_reduction: bool = True, min_stable_antecedents: int = 1, min_flexible_antecedents: int = 1, max_stable_antecedents: int = 5, max_flexible_antecedents: int = 5) method of actionrules.actionRulesDiscovery.actionRulesDiscovery.ActionRulesDiscovery instance
    Get action rules.
    Define antecedent and consequent.
    - stable_antecedents - List of column names.
    - flexible_antecedents - List of column names.
    - consequent - Name of the consequent column.
    Confidence and support.
    - conf - Value in % for minimal confidence in classification rules.
             For example, 60.
    - supp - Value in % for minimal support of classification rules.
             For example, 5.
    Desired clas

## Fit model to training data


In our example, we will focus on finding rules that provide actionable advise to passengers who have not survived.


In [5]:
actionRulesDiscovery.load_pandas(dataFrame)
actionRulesDiscovery.fit(stable_antecedents = ["Age", "Sex"],
            flexible_antecedents = ["Embarked", "Fare", "Pclass"],
            consequent = "Survived",
            conf=55,
            supp=3,
            desired_classes = ["1.0"],
)

#### Explanation of the settings

    consequent = "Survived"
    
In our example, we will focus on finding rules that provide actionable advise to passengers who have not survived.

    desired_classes = ["1.0"]
We want to find rules which will provide advise on actions, which will result in change of classifiction to survived (which is coded as 1.0 in data). Since the only other value of the class variable is 0.0 (Not survived), the learning will focus on finding rules that change the classification from `not survived` to `survived`.

    supp=3
We will discover action rules from classification rules that have a minimum support of 3%.
What is a support of a rule? Consider classification rule `IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=YES`. Our complete dataset covers 1310 passengers. Out of these, there are 198 passangers who match the complete rule (surviving females boarded in Southampton). The support of this rule is thus 198/1310= 15.1%.

    conf=55

We will discover action rules from classification rules that have a minimum confidence of 55%.
What is confidence of a rule? Consider classification rule `IF Boarded in Southampton AND Gender is Female AND Class is Third THEN survival=NO` Assuming that in the data, there is 291 passangers who match the conditions of this rule (boarded in  Southampton and are females). Out of this group, 198 passengers have survived. The confidence of the rule is thus  60/100 = 68%.

    stable_antecedents = ["Age", "Sex"]    

For all attributes in the passed dataset, we need to define whether they are stable or actionable. 
"Age" and "Sex" are designated as stable, because they cannot be changed -  an action rule proposing change in sex or age would be useless.

    flexible_antecedents = ["Embarked", "Fare", "Pclass"]   

As flexible (actionable) attributes we use "Embarked", "Fare", "Pclass". Flexible attributes allow changing of their state (an action can be taken). 

#### Number of discovered action rules

In [6]:
len(actionRulesDiscovery.get_pretty_action_rules())

13

## Interpreting the results

### Obtaining all discovered action rules

In [7]:
for rule in actionRulesDiscovery.get_action_rules_representation():
    print(rule)
    print(" ")

r = [(Sex: female) ∧ (Embarked: S → C)  ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.05267175572519084 and confidence: 0.5876187356698329
 
r = [(Age: <16.13336;32.10002)) ∧ (Embarked: S → C) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04198473282442748 and confidence: 0.40990990990991
 
r = [(Age: <16.13336;32.10002)) ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04732824427480916 and confidence: 0.5261127825349071
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: avg → very high) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.044274809160305344 and confidence: 0.43256997455470736
 
r = [(Age: <16.13336;32.10002)) ∧ (Pclass: 2.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.04732824427480916 and confidence: 0.42040850078557845
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: avg → very high)  ∧ (Pclass: 2.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.03969465648854962 and confidence: 0.46254681647940077
 
r = [(Age: <16.13336;32.10002)) ∧ (Fare: very low → ver

## Interpretation of a selected action rule  - condensed notation

We will analyze the fist rule on the output:

r = [(Sex: female) ∧ (Embarked: S → C)  ∧ (Pclass: 3.0 → 1.0) ] ⇒ [Survived: 0.0 → 1.0] with support: 0.05267175572519084 and confidence: 0.5876187356698329 

A single action rule can actually **be decomposed to two rules**, one representing the state *before* and the second one *after* the intervention.

#### Rule before
r = [(Sex: female) ∧ (Embarked: S)  ∧ (Pclass: 3.0) ] ⇒ [Survived: 0.0] 


#### Rule after
r = [(Sex: female) ∧ (Embarked: C)  ∧ (Pclass: 1.0) ] ⇒ [Survived: 1.0] 

The support and confidence of Rule before/after do not match support and confidence of the action rule. These detailed measures can be obtained with `get_action_rules()` (cf. last section below for an example).

### Parts of  action rule

##### Antecedent of the rule 
    [(Sex: female) ∧ (Embarked: S → C) ∧ (Pclass: 3.0 → 1.0) ] 

#### Consequent of the rule
     [Survived: 0.0 → 1.0] 

#### Measures of quality of the rule
     Support Before 0.06
     Support After 0.05
     Support of Action Rule: 0.05
     Confidence Before: 0.60
     Confidence After: 0.97
     Confidence of Action Rule: 0.59

The measures of quality show the quality of both parts of the rule (Before and After) and also the quality of the whole. 

Support of the action rule r that was extracted from classification rules r1 (Before) and r2 (After) is sup(r) = min(sup(r1), sup(r2)). It explains the percent of passengers the rule applies for.

Confidence of the action rule r that was extracted from classification rules r1 (Before) and r2 (After) is
r = conf(r1) ∗ conf(r2). It explains how many percent of female passengers, who would change their boarding place from Southampton to Cherbourg, and third-class to first-class, would have a different final state from did not survive to did survive.

### Individual conditions

     Sex: female

This rule applies to passangers, who are females,  


    Embarked: S
who boarded in Southampton, and 

    Pclass: 3.0
who travelled in the third class.

If these passengers would

    Embarked: S → C
change the boarding place to Cherbourg and at the same time
    
    Pclass: 3.0 → 1.0
change the passanger class from third class to first class

    [Survived: 0.0 → 1.0]
it will become more likely that they would survive,

    support: 0.05267175572519084 and confidence: 0.5876187356698329
It will apply for 5% of passangers with success rate 59%.
    

## Interpretation of a selected action rule  - textual notation

A selected action rule can also be represented in a self-explanatory, yet a bit more verbose, textual notation:

In [8]:
actionRulesDiscovery.get_pretty_action_rules()[0]

"If attribute 'Sex' is 'female', attribute 'Embarked' value 'S' is changed to 'C', attribute 'Pclass' value '3.0' is changed to '1.0', then 'Survived' value '0.0' is changed to '1.0' with support: 0.05267175572519084 and confidence: 0.5876187356698329."

## Interpretation of a selected action rule  - free text

A selected action rule can also be represented in a friendlier, yet a bit more verbose, textual notation:

**Women** who traveled from **Southampton** in the **third class** would increase their chance for survival if they would change their boarding place to **Cherbourg** and paid extra money for **first class**.

This notation cannot currently be generated algorithmically.

## How it works

### First action rule - before
The following code displays data that show instances in data that match the conditions of the "before" part of the action rule.

- Yellow background - stable attributes

- Orange background - flexible attributes

- Red text - Target attribute, unwanted state

- Green text - Target attribute, desired state

Note that the color highlighting does not work on Github.

You can notice that "Not survived" is the prevalent value.

In [9]:
actionRulesDiscovery.get_source_data_for_ar(0, True)

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
603,604,<32.10002;48.06668),S,avg,3.0,female,1.0
604,605,<0.1667;16.13336),S,very low,3.0,female,1.0
610,611,<32.10002;48.06668),S,lower,3.0,female,0.0
612,613,<16.13336;32.10002),S,lower,3.0,female,1.0
621,622,<16.13336;32.10002),S,lower,3.0,female,1.0
623,624,<0.1667;16.13336),S,higher,3.0,female,0.0
624,625,<0.1667;16.13336),S,higher,3.0,female,0.0
625,626,<16.13336;32.10002),S,lower,3.0,female,1.0
626,627,<32.10002;48.06668),S,very low,3.0,female,0.0
627,628,<0.1667;16.13336),S,higher,3.0,female,0.0


### First action rule - after
The following code displays data that show instances in data that match the conditions of the "after" part of the action rule.

- Yellow background - stable attributes

- Orange background - flexible attributes

- Red text - Target attribute, unwanted state

- Green text - Target attribute, desired state

Notice that nearly all instances have the outcome Survived=1.0, which indicates survival.

In [10]:
actionRulesDiscovery.get_source_data_for_ar(0, False)

,ID,Age,Embarked,Fare,Pclass,Sex,Survived
11,12,<16.13336;32.10002),C,very high,1.0,female,1.0
12,13,<16.13336;32.10002),C,very high,1.0,female,1.0
17,18,<48.06668;64.03334),C,very high,1.0,female,1.0
18,19,<16.13336;32.10002),C,very high,1.0,female,1.0
23,24,<32.10002;48.06668),C,very high,1.0,female,1.0
27,28,<16.13336;32.10002),C,very high,1.0,female,1.0
35,36,<32.10002;48.06668),C,very high,1.0,female,1.0
41,42,<32.10002;48.06668),C,higher,1.0,female,1.0
43,44,<48.06668;64.03334),C,very high,1.0,female,1.0
44,45,<32.10002;48.06668),C,very high,1.0,female,1.0


### Prediction
In the context of action rule mining, prediction means finding action rules that matches a given instance. Based on these rules, recommendations are generated.

Consider the following test instance:

In [11]:
new_data = [['<32.10002;48.06668)','female', 'S', 'very high' ,'3.0']] 
df_new_data = pd.DataFrame(new_data, columns = ["Age", "Sex", "Embarked", "Fare", "Pclass"]) 

The prediction is invoked as follows: 

In [12]:
actionRulesDiscovery.predict(df_new_data)

,Age,Embarked,Embarked-recommended,Fare,Pclass,Pclass-recommended,Sex,action rule,action rule target,support before,support after,action rule support,confidence before,confidence after,action rule confidence
0,<32.10002;48.06668),S,C,very high,3.0,1.0,female,0.0,1.0,0.059542,0.052672,0.052672,0.604651,0.971831,0.587619
0,<32.10002;48.06668),S,NaN,very high,3.0,1.0,female,10.0,1.0,0.061832,0.051145,0.051145,0.843750,0.626168,0.528329


Based on this table, the decision maker can choose one of the proposed courses of action.

There is one row in the resulting table for each matching action rule. The ID of the action rule is listed in the last column. For example, the first row corresponds to recommendations of Action rule with ID=0. This rule recommends to change the embarkement to Cherbourg and class to the first class. 

Imagine that the ship Titanic II would be created and ready for the first journey. It would
copy the cruise of the first Titanic. The shipbuilding company would sell tickets over the
Internet. In some cases, the system could recommend different options to increase the probability
of survival,

![Recommended actions](img/titanic.png)


### Machine representation

In [13]:
actionRulesDiscovery.get_action_rules()

[[[[['Sex', ('female',)]],
   [['Embarked', ('S', 'C')], ['Pclass', ('3.0', '1.0')]],
   ['Survived', ['0.0', '1.0']]],
  [0.059541984732824425, 0.05267175572519084, 0.05267175572519084],
  [0.6046511627906976, 0.971830985915493, 0.5876187356698329]],
 [[[['Age', ('<16.13336;32.10002)',)]],
   [['Embarked', ('S', 'C')]],
   ['Survived', ['0.0', '1.0']]],
  [0.2083969465648855, 0.04198473282442748, 0.04198473282442748],
  [0.6707616707616708, 0.6111111111111112, 0.40990990990991]],
 [[[['Age', ('<16.13336;32.10002)',)]],
   [['Pclass', ('3.0', '1.0')]],
   ['Survived', ['0.0', '1.0']]],
  [0.16793893129770993, 0.04732824427480916, 0.04732824427480916],
  [0.738255033557047, 0.7126436781609196, 0.5261127825349071]],
 [[[['Age', ('<16.13336;32.10002)',)]],
   [['Fare', ('avg', 'very high')]],
   ['Survived', ['0.0', '1.0']]],
  [0.0648854961832061, 0.044274809160305344, 0.044274809160305344],
  [0.648854961832061, 0.6666666666666666, 0.43256997455470736]],
 [[[['Age', ('<16.13336;32.10002

The output is a list of action rules. Each action rule is a list where the first part is an action rule itself, and the second part is a tuple of (support before, support after, action rule support) and (confidence before, confidence after, action rule confidence).